In [6]:
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [7]:
# load dataset
df = pd.read_csv("./data/sms_classifications.csv")
df.head()

,sms,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [8]:
test_data = df['sms'][:5]

test_data

0    Go until jurong point, crazy.. Available only ...
1                      Ok lar... Joking wif u oni...\n
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: sms, dtype: object

In [9]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8501/v1/models/sms_spam_classifications_model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1735269485'}]}


In [10]:
def prepare_json(text):
    feature_spec = {
        "sms": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [11]:
endpoint = "http://localhost:8501/v1/models/sms_spam_classifications_model:predict"

In [12]:
def make_predictions(text):
    json_data = prepare_json(text)
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()
    if prediction:
        prediction_value = prediction['predictions'][0][0]
        result = "Spam" if prediction_value > 0.5 else "Not Spam"
    else:
        result = "Error: No predictions found in response."

    print(result)

In [13]:
sms_test = df['sms'][2]

In [14]:
make_predictions(sms_test)

Spam
